In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

In [ ]:
import numpy as np
import rioxarray
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

In [ ]:
# product definition 
product_path = (
    "data/S1A_IW_SLC__1SDV_20211223T170557_20211223T170624_041139_04E360_B8E2.SAFE"
)
dem_urlpath = "data/Gran-Sasso-3m-DEM-small.tif"
measurement_group = "IW3/VV"

In [ ]:
apps.backward_geocode_sentinel1(product_path, measurement_group, dem_urlpath, correct_radiometry=False)
geocoded_beta0 = rioxarray.open_rasterio("GRD.tif")
geocoded_beta0.plot(vmax=5, vmin=0)

In [ ]:
apps.backward_geocode_sentinel1(product_path, measurement_group, dem_urlpath, correct_radiometry=True, output_urlpath="GRDC.tif")
geocoded_beta0_c = rioxarray.open_rasterio("GRDC.tif")
geocoded_beta0_c.plot(vmax=5, vmin=0)

## CHECK INTERNAL FUNCTIONS

### READ ORBIT AND INTERPOLATE

In [ ]:
orbit_ecef = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{measurement_group}/orbit"
)
orbit_interpolator = orbit.OrbitPolyfitIterpolator.from_position(orbit_ecef.position)
azimuth_time = orbit_interpolator.azimuth_time_range()

position_sar = orbit_interpolator.position(azimuth_time)
velocity_sar = orbit_interpolator.velocity(azimuth_time)

direction_sar = velocity_sar / np.linalg.norm(velocity_sar)

In [ ]:
dem = scene.open_dem_raster(dem_urlpath)

In [ ]:
try:
    dem_ecef = scene.convert_to_dem_ecef(dem)
except:
    # first time we run it we get a GDAL exception :shrug:
    dem_ecef = scene.convert_to_dem_ecef(dem)

### BACKWARD GEOCODING DEM and DEM_CENTERS

In [ ]:
dem_coords = geocoding.backward_geocode(dem_ecef, position_sar, direction_sar)

### COMPUTE GAMMA WEIGHTS

In [ ]:
measurement = xr.open_dataarray(product_path, group=measurement_group, engine="sentinel-1")

In [ ]:
azimuth_time0 = measurement.azimuth_time[0] 
slant_range_time0 = measurement.slant_range_time[0]

In [ ]:
%%time
weights = geocoding.gamma_weights(
    dem_ecef, 
    dem_coords,
    slant_range_time0=slant_range_time0.data,
    azimuth_time0=azimuth_time0.data,
    slant_range_time_interval=measurement.attrs["slant_range_time_interval"],
    azimuth_time_interval=measurement.attrs["azimuth_time_interval"],
    pixel_spacing_azimuth=measurement.attrs["sar:pixel_spacing_azimuth"],
    pixel_spacing_range=measurement.attrs["sar:pixel_spacing_range"],
)

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=3, figsize=(30, 15))

_ = geocoded_beta0.plot(ax=axes[0], vmax=5)
axes[0].grid(c="black")

_ = (geocoded_beta0 / weights).plot(ax=axes[1], vmax=5)
axes[1].grid(c="black")

_ = weights.transpose("y", "x").plot(ax=axes[2], vmax=5)
axes[2].grid(c="black")